In [1]:
import pandas as pd
import numpy as np
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from sklearn.preprocessing import StandardScaler
import pickle
import matplotlib.pyplot as plt

##### 加载数据

def load_data(csv_path):
    df = pd.read_csv(csv_path)
    features = df.values
    #### 如果是差分处理过的数据可以不用做标准化
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    return torch.tensor(features_scaled, dtype=torch.float32), scaler


##### SSM模型

def ssm_model_train(y, state_dim=5):

    T, obs_dim = y.shape
    ######  恢复原始A的初始化（0.95）
    A = pyro.param("A", torch.eye(state_dim) * 0.95)
    C = pyro.param("C", torch.randn(obs_dim, state_dim) * 0.1)
    ######  状态噪声协方差
    Q_diag = pyro.param("Q_diag", torch.ones(state_dim) * 0.1, constraint=dist.constraints.positive)
    Q = torch.diag(Q_diag)
    ######  观测噪声协方差（恢复原始0.1）
    R_diag = pyro.param("R_diag", torch.ones(obs_dim) * 0.1, constraint=dist.constraints.positive)
    R = torch.diag(R_diag)
    
    ######  初始状态
    z = pyro.sample("z0", dist.MultivariateNormal(torch.zeros(state_dim), torch.eye(state_dim)))
    ######  递归生成状态
    for t in range(1, T):
        z = pyro.sample(f"z{t}", dist.MultivariateNormal(A @ z, Q))
        pyro.sample(f"y{t}", dist.MultivariateNormal(C @ z, R), obs=y[t])
    return A, C, Q, R


######  卡尔曼滤波作为在线推断

def kalman_filter(y, A, C, Q, R, z0=None, P0=None):

    T, obs_dim = y.shape
    state_dim = A.shape[0]
    
    if z0 is None:
        z0 = torch.zeros(state_dim)
    if P0 is None:
        P0 = torch.eye(state_dim) * 0.1
    
    z = z0
    P = P0
    states = [z]
    
    for t in range(1, T):
        #### 预测步骤
        z_pred = A @ z
        P_pred = A @ P @ A.T + Q
        
        #### 更新步骤
        y_t = y[t]
        K = P_pred @ C.T @ torch.inverse(C @ P_pred @ C.T + R)
        z = z_pred + K @ (y_t - C @ z_pred)
        P = (torch.eye(state_dim) - K @ C) @ P_pred
        
        states.append(z)
    
    return torch.stack(states), P  ##### 返回状态序列和最后时刻的协方差



########## 模型保存和加载

def save_model(params, scaler, path):

    A, C, Q, R = params
    torch.save({
        "A": A,
        "C": C,
        "Q": Q,
        "R": R,
        "scaler": scaler
    }, path)
    print(f"模型已保存到 {path}")

def load_model(path):

    checkpoint = torch.load(path)
    A = checkpoint["A"]
    C = checkpoint["C"]
    Q = checkpoint["Q"]
    R = checkpoint["R"]
    scaler = checkpoint["scaler"]
    return (A, C, Q, R), scaler


######## 处理新数据，对单个bar生成SSM特征

def process_new_data(new_bar, model_params, scaler, last_z, last_P):

    A, C, Q, R = model_params
    ##### 标准化新数据
    new_bar_scaled = scaler.transform(new_bar.reshape(1, -1)).flatten()
    new_bar_tensor = torch.tensor(new_bar_scaled, dtype=torch.float32)
    
    ##### 预测步骤
    z_pred = A @ last_z
    P_pred = A @ last_P @ A.T + Q
    
    ##### 更新步骤
    K = P_pred @ C.T @ torch.inverse(C @ P_pred @ C.T + R)
    new_z = z_pred + K @ (new_bar_tensor - C @ z_pred)
    new_P = (torch.eye(A.shape[0]) - K @ C) @ P_pred
    
    return new_z, new_P



########  模型训练保存

def main():
    ##### 参数配置
    config = {
        "csv_path": "./test.csv",    ######### 原始特征文件
        "model_path": "ssm_realtime_model.pt",      ######### 模型保存
        "feature_save_path": "ssm_features.csv",  ######### 特征保存
        "state_dim": 5,   ######### 新特征数据维度数量
        "epochs": 50  ######### 训练轮次
    }
    
    ##### 加载数据
    y, scaler = load_data(config["csv_path"])
    T, obs_dim = y.shape
    print(f"数据加载完成：{T}行，{obs_dim}维特征")
    
    ##### 训练模型
    optimizer = pyro.optim.Adam({"lr": 0.01})
    svi = SVI(
        model=lambda: ssm_model_train(y, state_dim=config["state_dim"]),
        guide=pyro.infer.autoguide.AutoNormal(lambda: ssm_model_train(y, state_dim=config["state_dim"])),
        optim=optimizer,
        loss=Trace_ELBO()
    )
    print("训练模型...")
    for epoch in range(config["epochs"]):
        loss = svi.step()
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{config['epochs']} | Loss: {loss / T:.4f}")
    
    ##### 提取模型参数
    A = pyro.param("A").detach()
    C = pyro.param("C").detach()
    Q = torch.diag(pyro.param("Q_diag").detach())
    R = torch.diag(pyro.param("R_diag").detach())
    model_params = (A, C, Q, R)
    
    ##### 用卡尔曼滤波生成特征
    states, _ = kalman_filter(y, A, C, Q, R)
    print(f"生成特征形状：{states.shape}（{states.shape[0]}行，{states.shape[1]}维）")
    
    
    ##### 保存生成的特征

    feature_df = pd.DataFrame(
        states.numpy(),  # 形状：[T, state_dim]
        columns=[f"ssm_feature_{i}" for i in range(config["state_dim"])]
    )
    feature_df.to_csv(config["feature_save_path"], index=False)
    print(f"特征已保存到 {config['feature_save_path']}")    
    
   
    
    ##### 保存模型
    save_model(model_params, scaler, config["model_path"])
    
    ##### 处理新数据
    last_idx = T - 1
    #####  获取最后时刻的状态和协方差
    _, last_P = kalman_filter(y[:last_idx+1], A, C, Q, R)
    last_z = states[last_idx]   ######## 最后时刻的状态
    
    ##### 新数据
    new_bar = y[last_idx].numpy()
    print(f"新数据：{new_bar.shape}")
    
    #### 实时生成新特征
    new_z, new_P = process_new_data(new_bar, model_params, scaler, last_z, last_P)
    print(f"新数据的SSM特征：{new_z.numpy()}")
    
    
    return states

if __name__ == "__main__":
    states = main()

数据加载完成：1457行，77维特征
训练模型...
Epoch 10/50 | Loss: 327.2564
Epoch 20/50 | Loss: 251.3681
Epoch 30/50 | Loss: 171.5892
Epoch 40/50 | Loss: 118.6649
Epoch 50/50 | Loss: 95.0418
生成特征形状：torch.Size([1457, 5])（1457行，5维）
特征已保存到 ssm_features.csv
模型已保存到 ssm_realtime_model.pt
新数据：(77,)
新数据的SSM特征：[ 12.618005 -33.169773   6.282862 -10.705537 -13.069874]


In [ ]:
########### 实时特征输出 ###########

import pandas as pd
import numpy as np
import torch
import pyro
import pyro.distributions as dist
from sklearn.preprocessing import StandardScaler
import pickle


### 加载模型
model_params, scaler = load_model("ssm_realtime_model.pt")
A, C, Q, R = model_params

### 初始化状态（使用训练时的最后状态）
### 从历史数据计算
y_history, _ = load_data("ssm_features.csv")  ##### 加载训练数据
_, last_P = kalman_filter(y_history, A, C, Q, R)  ##### 计算最后时刻的协方差
last_z = torch.zeros(5)  ##### 实际应从历史数据获取，这里简化为全零向量

### 实时接收新bar并生成特征
while True:
    new_bar = get_new_bar()  ##### 实盘函数，获取新的特征
    new_z, new_P = process_new_data(new_bar, model_params, scaler, last_z, last_P)
    
    ##### 用new_z作为当前bar的SSM特征
    predict_with_xgboost(new_z.numpy())
    
    ##### 更新状态（用于下一个bar）
    last_z, last_P = new_z, new_P